# Автоматичне опрацювання української мови

## 1. Сегментація тексту

Прочитаймо текст "Тигроловів":

In [1]:
with open("data/tyhrolovy.txt", "r") as f:
    tyhrolovy_raw = f.read()

print("У тексті є {} символів.".format(len(tyhrolovy_raw)))

У тексті є 476213 символів.


In [2]:
print(tyhrolovy_raw[:200])

Тигролови
Іван Багряний
ЧАСТИНА ПЕРША

Розділ перший

ДРАКОН

...Вирячивши вогненні очі, дихаючи полум'ям і димом, потрясаючи ревом пустелі і нетра і вогненним хвостом замітаючи слід, летів дракон.

Н


**Сирий текст** варто сегментувати на абзаци, речення, слова. Для цього використаємо бібліотеку [tokenize_uk](https://github.com/lang-uk/tokenize-uk).

Як ви гадаєте:
- які бувають складнощі автоматичного розбиття тексту на речення?
- які бувають складнощі автоматичного розбиття тексту на слова?
- які бувають складнощі автоматичного розбиття тексту на абзаци?

In [3]:
import tokenize_uk

In [4]:
tyhrolovy_words = tokenize_uk.tokenize_words(tyhrolovy_raw)

print("У тексті є {} слів.".format(len(tyhrolovy_words)))
print()
print(tyhrolovy_words[:40])

У тексті є 99709 слів.

['Тигролови', 'Іван', 'Багряний', 'ЧАСТИНА', 'ПЕРША', 'Розділ', 'перший', 'ДРАКОН', '...', 'Вирячивши', 'вогненні', 'очі', ',', 'дихаючи', "полум'ям", 'і', 'димом', ',', 'потрясаючи', 'ревом', 'пустелі', 'і', 'нетра', 'і', 'вогненним', 'хвостом', 'замітаючи', 'слід', ',', 'летів', 'дракон', '.', 'Не', 'з', 'китайських', 'казок', 'і', 'не', 'з', 'пагод']


In [5]:
tyhrolovy_tokenized = tokenize_uk.tokenize_text(tyhrolovy_raw)

for i in tyhrolovy_tokenized[:8]:
    print(i)

[['Тигролови']]
[['Іван', 'Багряний']]
[['ЧАСТИНА', 'ПЕРША']]
[['Розділ', 'перший']]
[['ДРАКОН']]
[['...', 'Вирячивши', 'вогненні', 'очі', ',', 'дихаючи', "полум'ям", 'і', 'димом', ',', 'потрясаючи', 'ревом', 'пустелі', 'і', 'нетра', 'і', 'вогненним', 'хвостом', 'замітаючи', 'слід', ',', 'летів', 'дракон', '.']]
[['Не', 'з', 'китайських', 'казок', 'і', 'не', 'з', 'пагод', 'Тібету', '—', 'він', 'знявся', 'десь', 'з', 'громохкого', 'центру', 'країни', '"', 'чудес', '"', ',', 'вилетів', 'з', 'чорного', 'пекла', 'землі', 'людоловів', 'і', 'гнав', 'над', 'просторами', '...'], ['Над', 'безмежжям', 'Уралу', '...'], ['Через', 'хащі', 'Сибіру', '...'], ['Через', 'грізний', ',', 'понурий', 'Байкал', '...'], ['Через', 'дикі', 'кряжі', 'Зайбакалля', '...'], ['Через', 'Становий', 'хребет', ',', '—', 'звивався', 'над', 'ним', 'межи', 'скель', 'і', 'шпилів', '...'], ['Високо', 'в', 'небі', ',', 'сіючи', 'іскри', 'й', 'сморід', ',', 'летів', 'і', 'летів', 'у', 'безмежній', 'ночі', '.']]
[['...', 'Пала

## 2. Частиномовний аналіз

Поговорімо про складнощі автоматичного частиномовного аналізу:
- які слова можуть позначати різні частини мови?
- які частини мови може позначати словоформа *"край"*? *"прав"*? *"багатій"*?
- чи є неоднозначність у фразі *"коло друзів та незнайомців"*?

Більше прикладів:

<img src="img/ambiguous-pos.png" alt="Приклади слів, що позначають кілька частин мови" width="40%" height="40%" align="left"/>

Бібліотека [pymorphy2](https://github.com/kmike/pymorphy2) надає всі можливі варіанти розбору кожного слова, проте не визначає правильний розбір у контексті.

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [7]:
morph.parse("стіл")

[Parse(word='стіл', tag=OpencorporaTag('NOUN,inan masc,nomn'), normal_form='стіл', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стіл', 2710, 0),)),
 Parse(word='стіл', tag=OpencorporaTag('NOUN,inan masc,accs'), normal_form='стіл', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стіл', 2710, 5),))]

In [8]:
morph.parse("прав")

[Parse(word='прав', tag=OpencorporaTag('NOUN,inan plur,gent'), normal_form='право', score=1.0, methods_stack=((DictionaryAnalyzer(), 'прав', 22, 9),)),
 Parse(word='прав', tag=OpencorporaTag('VERB,impf sing,2per,impr'), normal_form='правити', score=1.0, methods_stack=((DictionaryAnalyzer(), 'прав', 302, 2),)),
 Parse(word='прав', tag=OpencorporaTag('VERB,impf masc,past'), normal_form='прати', score=1.0, methods_stack=((DictionaryAnalyzer(), 'прав', 571, 20),))]

In [9]:
morph.parse("багатій")

[Parse(word='багатій', tag=OpencorporaTag('ADJF,compb femn,datv'), normal_form='багатий', score=1.0, methods_stack=((DictionaryAnalyzer(), 'багатій', 76, 11),)),
 Parse(word='багатій', tag=OpencorporaTag('ADJF,compb femn,loct'), normal_form='багатий', score=1.0, methods_stack=((DictionaryAnalyzer(), 'багатій', 76, 14),)),
 Parse(word='багатій', tag=OpencorporaTag('VERB,impf sing,2per,impr'), normal_form='багатіти', score=1.0, methods_stack=((DictionaryAnalyzer(), 'багатій', 121, 2),)),
 Parse(word='багатій', tag=OpencorporaTag('NOUN,anim masc,nomn'), normal_form='багатій', score=1.0, methods_stack=((DictionaryAnalyzer(), 'багатій', 128, 0),))]

In [10]:
# Випробуйте інші слова. Гляньте на аналіз багатозначних слів ("до", "того") та нестандартних слів ("євробляха").

#### Як із цим працювати?

In [11]:
morph.parse("багатій")[0]

Parse(word='багатій', tag=OpencorporaTag('ADJF,compb femn,datv'), normal_form='багатий', score=1.0, methods_stack=((DictionaryAnalyzer(), 'багатій', 76, 11),))

In [12]:
morph.parse("багатій")[0].normal_form

'багатий'

In [13]:
morph.parse("багатій")[0].tag

OpencorporaTag('ADJF,compb femn,datv')

In [14]:
morph.parse("багатій")[0].tag.POS

'ADJF'

In [15]:
morph.parse("багатій")[0].tag.gender

'femn'

In [16]:
morph.parse("багатій")[0].tag.animacy

In [17]:
morph.parse("багатій")[3].tag.animacy

'anim'

Погляньмо на леми першого речення "Тигроловів":

In [18]:
for word in tyhrolovy_words[9:32]:
    parsed_word = morph.parse(word)[0]
    print("{:10}\t{}".format(word, parsed_word.normal_form))

Вирячивши 	вирячивши
вогненні  	вогненний
очі       	око
,         	,
дихаючи   	дихаючи
полум'ям  	полум'я
і         	і
димом     	дим
,         	,
потрясаючи	потрясаючи
ревом     	рев
пустелі   	пустеля
і         	і
нетра     	нетра
і         	і
вогненним 	вогненний
хвостом   	хвіст
замітаючи 	замітаючи
слід      	слід
,         	,
летів     	летіти
дракон    	дракон
.         	.


In [19]:
for word in tyhrolovy_words[9:32]:
    parsed_word = morph.parse(word)[0]
    print("{:10}\t{:10}\t{}".format(word, parsed_word.normal_form, parsed_word.tag.POS))

Вирячивши 	вирячивши 	GRND
вогненні  	вогненний 	ADJF
очі       	око       	NOUN
,         	,         	None
дихаючи   	дихаючи   	GRND
полум'ям  	полум'я   	NOUN
і         	і         	CONJ
димом     	дим       	NOUN
,         	,         	None
потрясаючи	потрясаючи	GRND
ревом     	рев       	NOUN
пустелі   	пустеля   	NOUN
і         	і         	CONJ
нетра     	нетра     	None
і         	і         	CONJ
вогненним 	вогненний 	ADJF
хвостом   	хвіст     	NOUN
замітаючи 	замітаючи 	GRND
слід      	слід      	NOUN
,         	,         	None
летів     	летіти    	VERB
дракон    	дракон    	NOUN
.         	.         	None


## 3. Частотний аналіз тексту

#### Проведімо аналіз усіх слів у творах *"Тигролови"* та *"Собор"*:

In [20]:
with open("data/sobor.txt", "r") as f:
    sobor_raw = f.read()
    sobor_words = tokenize_uk.tokenize_words(sobor_raw)

print("У тексті є {} слів.".format(len(sobor_words)))

У тексті є 111067 слів.


In [21]:
corpus_t = [morph.parse(word)[0] for word in tyhrolovy_words]

In [22]:
corpus_s = [morph.parse(word)[0] for word in sobor_words]

#### Як знайти найчастотніші слова в романі?

In [23]:
from collections import Counter

def most_freq(corpus, n):
    "Return the most common words."
    words = Counter()
    for word in corpus:
        words[word.word] += 1
    return words.most_common(n)

In [24]:
most_freq(corpus_t, 20)

[(',', 10124),
 ('.', 5783),
 ('і', 3134),
 ('—', 2840),
 ('...', 1516),
 ('на', 1484),
 ('не', 1325),
 ('а', 1246),
 ('"', 1215),
 ('в', 1182),
 ('з', 1080),
 ('як', 944),
 ('!', 867),
 ('що', 836),
 ('та', 755),
 ('й', 663),
 ('-', 598),
 ('у', 537),
 ('до', 529),
 ('він', 521)]

Як можна покращити функцію **most_freq**?

In [25]:
def most_freq(corpus, n):
    "Return the most common lemmas."
    words = Counter()
    for word in corpus:
        if word.tag.POS in {"NOUN", "VERB", "ADJF"}:
            words[word.normal_form] += 1
    return words.most_common(n)

In [26]:
most_freq(corpus_t, 20)

[('бути', 703),
 ('до', 529),
 ('григорій', 423),
 ('йога', 316),
 ('наталка', 225),
 ('око', 224),
 ('старий', 214),
 ('про', 186),
 ('рука', 174),
 ('дивитися', 158),
 ('знати', 152),
 ('піти', 151),
 ('бачити', 149),
 ('голова', 144),
 ('стояти', 143),
 ('хлопець', 135),
 ('йти', 129),
 ('тога', 123),
 ('сірко', 123),
 ('мати', 122)]

In [27]:
COMMONLY_CONFUSED = {'бути', 'до', 'йога', 'мен', 'перед', 'про', 'тога'}

def most_freq(corpus, n):
    "Return the most common lemmas."
    words = Counter()
    for word in corpus:
        if word.tag.POS in {"NOUN", "VERB", "ADJF"} and word.normal_form not in COMMONLY_CONFUSED:
            words[word.normal_form] += 1
    return words.most_common(n)

In [28]:
most_freq(corpus_t, 20)

[('григорій', 423),
 ('наталка', 225),
 ('око', 224),
 ('старий', 214),
 ('рука', 174),
 ('дивитися', 158),
 ('знати', 152),
 ('піти', 151),
 ('бачити', 149),
 ('голова', 144),
 ('стояти', 143),
 ('хлопець', 135),
 ('йти', 129),
 ('сірко', 123),
 ('мати', 122),
 ('раз', 120),
 ('світ', 117),
 ('грицько', 115),
 ('гриць', 114),
 ('вода', 113)]

In [29]:
most_freq(corpus_s, 20)

[('собор', 289),
 ('єлька', 243),
 ('час', 195),
 ('життя', 189),
 ('люди', 168),
 ('мати', 162),
 ('сказати', 157),
 ('око', 157),
 ('лобода', 157),
 ('рука', 149),
 ('старий', 144),
 ('тобі', 141),
 ('знати', 139),
 ('баглай', 133),
 ('бачити', 130),
 ('ніч', 129),
 ('зараза', 123),
 ('людина', 116),
 ('небо', 115),
 ('треба', 114)]

#### Як знайти найчастотніші колокації в романі?

In [30]:
def most_freq_colloc(corpus, n):
    "Return the most common adjective-noun collocations."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [31]:
most_freq_colloc(corpus_t, 20)

[(('старий', 'сірко'), 84),
 (('старий', 'мороз'), 10),
 (('дівочий', 'голос'), 9),
 (('цілий', 'світ'), 7),
 (('осяяний', 'сонце'), 6),
 (('скажений', 'поїзд'), 6),
 (('подібний', 'до'), 6),
 (('золотий', 'ельдорадо'), 6),
 (('добрий', 'гумор'), 6),
 (('старий', 'сірчиха'), 6),
 (('голубий', 'падь'), 6),
 (('вогненний', 'хвіст'), 5),
 (('мерехтливий', 'око'), 5),
 (('блідий', 'обличчя'), 5),
 (('великий', 'начальник'), 5),
 (('шостий', 'частина'), 5),
 (('другий', 'кінець'), 5),
 (('височенний', 'кедр'), 5),
 (('другий', 'бока'), 5),
 (('другий', 'день'), 5)]

In [32]:
most_freq_colloc(corpus_s, 20)

[(('цілий', 'ніч'), 12),
 (('старий', 'металург'), 9),
 (('цілий', 'день'), 7),
 (('протилежний', 'берег'), 7),
 (('зоряний', 'вода'), 6),
 (('рідний', 'батько'), 6),
 (('робочий', 'місце'), 5),
 (('бурий', 'небо'), 4),
 (('український', 'металург'), 4),
 (('робітничий', 'передмістя'), 4),
 (('заводський', 'дим'), 4),
 (('козацький', 'собор'), 4),
 (('водний', 'станція'), 4),
 (('останній', 'час'), 4),
 (('старий', 'лобода'), 4),
 (('наступний', 'день'), 4),
 (('великий', 'луг'), 4),
 (('старий', 'обер'), 4),
 (('старий', 'люди'), 4),
 (('вічний', 'абсолют'), 4)]

In [33]:
def most_freq_colloc_for_noun(corpus, n, noun):
    "Return the most common adjective-noun collocations with a specified noun."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.normal_form == noun and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [34]:
most_freq_colloc_for_noun(corpus_t, 20, "дівчина")

[(('вражий', 'дівчина'), 2),
 (('булий', 'дівчина'), 2),
 (('бистроокий', 'дівчина'), 1),
 (('ставний', 'дівчина'), 1),
 (('гостроязикий', 'дівчина'), 1),
 (('замріяний', 'дівчина'), 1),
 (('химерний', 'дівчина'), 1),
 (('вродливий', 'дівчина'), 1),
 (('лісовий', 'дівчина'), 1),
 (('гарний', 'дівчина'), 1),
 (('чорнявий', 'дівчина'), 1),
 (('вимучений', 'дівчина'), 1),
 (('чистий', 'дівчина'), 1),
 (('незрозумілий', 'дівчина'), 1),
 (('балакучий', 'дівчина'), 1),
 (('бідолашний', 'дівчина'), 1)]

In [35]:
most_freq_colloc_for_noun(corpus_s, 20, "дівчина")

[(('осиротілий', 'дівчина'), 1),
 (('перспективний', 'дівчина'), 1),
 (('сільський', 'дівчина'), 1),
 (('розкудланий', 'дівчина'), 1)]

In [36]:
# Спробуйте інші іменники, прикметники чи контексти.

In [37]:
def most_freq_colloc_for_adj(corpus, n, adj):
    "Return the most common adjective-noun collocations with a specified adjective."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and prev_word.normal_form == adj:
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [38]:
most_freq_colloc_for_adj(corpus_t, 20, "добрий")

[(('добрий', 'гумор'), 6),
 (('добрий', 'настрій'), 3),
 (('добрий', 'бінокль'), 2),
 (('добрий', 'дерево'), 1),
 (('добрий', 'кінь'), 1),
 (('добрий', 'ічага'), 1),
 (('добрий', 'кожух'), 1),
 (('добрий', 'обід'), 1),
 (('добрий', 'пес'), 1),
 (('добрий', 'гість'), 1),
 (('добрий', 'кім'), 1),
 (('добрий', 'приятель'), 1),
 (('добрий', 'хлопець'), 1),
 (('добрий', 'жовтяк'), 1),
 (('добрий', 'лошак'), 1),
 (('добрий', 'вдача'), 1)]

In [39]:
most_freq_colloc_for_adj(corpus_s, 20, "добрий")

[(('добрий', 'настрій'), 2),
 (('добрий', 'ранка'), 1),
 (('добрий', 'наждак'), 1),
 (('добрий', 'клімат'), 1),
 (('добрий', 'виробничник'), 1),
 (('добрий', 'хутір'), 1),
 (('добрий', 'тряска'), 1),
 (('добрий', 'гумор'), 1)]

In [40]:
def most_freq_colloc_anim(corpus, n):
    "Return the most common adjective-noun collocations with animate nouns."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and word.tag.animacy == "anim" and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [41]:
most_freq_colloc_anim(corpus_t, 20)

[(('старий', 'сірко'), 84),
 (('великий', 'начальник'), 5),
 (('сонячний', 'зайчик'), 4),
 (('цибатий', 'зять'), 4),
 (('малий', 'дитина'), 3),
 (('якутський', 'пес'), 3),
 (('вражий', 'мама'), 3),
 (('безумний', 'сміливець'), 2),
 (('державний', 'злочинець'), 2),
 (('земний', 'кулі'), 2),
 (('владивостоцький', 'турок'), 2),
 (('дивний', 'бог'), 2),
 (('змучений', 'мандрівник'), 2),
 (('вражий', 'дівчина'), 2),
 (('знаменитий', 'мисливець'), 2),
 (('бісовий', 'віра'), 2),
 (("нав'ючений", 'кінь'), 2),
 (('природний', 'солон'), 2),
 (('рясний', 'дуб'), 2),
 (('старий', 'мороз'), 2)]

In [42]:
most_freq_colloc_anim(corpus_s, 20)

[(('старий', 'металург'), 9),
 (('рідний', 'батько'), 6),
 (('український', 'металург'), 4),
 (('старий', 'люди'), 4),
 (('старший', 'брат'), 3),
 (('сліпий', 'танкіст'), 3),
 (('заслужений', 'металург'), 3),
 (('четвертий', 'домна'), 3),
 (('відповідальний', 'товариш'), 3),
 (('молодий', 'архітектор'), 3),
 (('сучасний', 'людина'), 3),
 (('білий', 'кінь'), 3),
 (('плавневий', 'лелеко'), 2),
 (('молодий', 'дружина'), 2),
 (('контрактований', 'студент'), 2),
 (('старий', 'баглаїса'), 2),
 (('золотий', 'зірка'), 2),
 (('найтяжчий', 'робот'), 2),
 (('перший', 'домна'), 2),
 (('двоюрідний', 'брат'), 2)]

In [43]:
# Порівняйте колокації для іменників жіночого роду з колокаціями для іменників чоловічого роду.

In [44]:
def most_freq_prep_adj_noun(corpus, n):
    "Return the most common prep-adj-noun collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "NOUN" and corpus[i-1].tag.POS == "ADJF" and corpus[i-2].tag.POS == "PREP":
            collocs[(corpus[i-2].word, corpus[i-1].word, word.word)] += 1
    return collocs.most_common(n)

In [45]:
most_freq_prep_adj_noun(corpus_t, 20)

[(('з', 'блискавичною', 'швидкістю'), 4),
 (('з', 'другого', 'боку'), 4),
 (('зі', 'скаженого', 'поїзда'), 2),
 (('біля', 'крайнього', 'столика'), 2),
 (('в', 'цілім', 'експресі'), 2),
 (('у', 'другім', 'місці'), 2),
 (('із', 'скаженого', 'поїзда'), 2),
 (('в', 'доброму', 'гуморі'), 2),
 (('у', 'вишиваних', 'сорочках'), 2),
 (('у', 'старого', 'сірка'), 2),
 (('в', 'надійні', 'руки'), 2),
 (('в', 'оленячій', 'голові'), 2),
 (('з', 'непокритою', 'головою'), 2),
 (('у', 'цілому', 'світі'), 2),
 (('під', 'рясним', 'дубом'), 2),
 (('в', "кам'яних", 'розсипах'), 2),
 (('під', 'григорієвим', 'куреником'), 2),
 (('з', 'дикого', 'каменю'), 2),
 (('в', 'другий', 'бік'), 2),
 (('у', 'нові', 'шати'), 2)]

In [46]:
most_freq_prep_adj_noun(corpus_s, 20)

[(('без', 'єдиного', 'цвяха'), 3),
 (('від', 'найменшого', 'доторку'), 2),
 (('з', 'другого', 'боку'), 2),
 (('з', 'розірваними', 'ланцюгами'), 2),
 (('під', 'ліхтарним', 'стовпом'), 2),
 (('з', 'ягорового', 'двору'), 2),
 (('у', 'білій', 'сорочці'), 2),
 (('з', 'засуканими', 'рукавами'), 2),
 (('з', 'городньої', 'бригади'), 2),
 (('в', 'дозаводські', 'часи'), 1),
 (('з', 'бокових', 'бань'), 1),
 (('між', 'плавкими', 'обрисами'), 1),
 (('з', 'соборної', 'висоти'), 1),
 (('в', 'місячнім', 'сяйві'), 1),
 (('в', 'розлогих', 'опуклостях'), 1),
 (('по', 'зачіплянському', 'килиму'), 1),
 (('з', 'кетяжистих', 'акацій'), 1),
 (('у', 'мудрій', 'злагоді'), 1),
 (('в', 'місячнім', 'мареві'), 1),
 (('під', 'навислим', 'цвітом'), 1)]

In [47]:
def most_freq_verb_prep_noun(corpus, n):
    "Return the most common verb-prep-noun collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "NOUN" and corpus[i-1].tag.POS == "PREP" and corpus[i-2].tag.POS == "VERB":
            collocs[(corpus[i-2].normal_form, corpus[i-1].word, word.word)] += 1
    return collocs.most_common(n)

In [48]:
most_freq_verb_prep_noun(corpus_t, 20)

[(('зірватися', 'з', 'місця'), 4),
 (('дивитися', 'у', 'вікно'), 3),
 (('дивитися', 'у', 'воду'), 3),
 (('тримати', 'попід', 'руки'), 2),
 (('бити', 'в', 'очі'), 2),
 (('вийняти', 'з', 'кишені'), 2),
 (('зняти', 'зі', 'стіни'), 2),
 (('бавитися', 'з', 'ведмежам'), 2),
 (('пірнути', 'в', 'нетрі'), 2),
 (('майнути', 'в', 'голові'), 2),
 (('бути', 'в', 'нім'), 2),
 (("з'їхати", 'з', 'глузду'), 2),
 (('хукати', 'в', 'небо'), 2),
 (('випити', 'по', 'чарці'), 2),
 (('вилетіти', 'з', 'чорного'), 1),
 (('гнати', 'над', 'просторами'), 1),
 (('палахкотіти', 'над', 'проваллями'), 1),
 (('звиватися', 'над', 'прірвами'), 1),
 (('пролітати', 'із', 'свистом'), 1),
 (('гнати', 'у', 'безвість'), 1)]

In [49]:
most_freq_verb_prep_noun(corpus_s, 20)

[(('повертатися', 'з', 'інституту'), 2),
 (('сидіти', 'у', 'човні'), 2),
 (('взяти', 'в', 'риштовання'), 2),
 (('бути', 'для', 'людини'), 2),
 (('сісти', 'в', 'машину'), 2),
 (('зайти', 'в', 'тінь'), 2),
 (('шубовснути', 'у', 'воду'), 2),
 (('залежати', 'від', 'того'), 2),
 (('мати', 'у', 'собі'), 2),
 (('носити', 'у', 'собі'), 2),
 (('відправити', 'в', 'союз'), 2),
 (('існувати', 'в', 'реальності'), 1),
 (('поринати', 'в', 'сон'), 1),
 (('стояти', 'над', 'селищами'), 1),
 (('бурхати', 'у', 'небо'), 1),
 (('повипадати', 'з', 'гнізда'), 1),
 (('підвестися', 'над', 'гніздом'), 1),
 (('стояти', 'з', 'щілинами'), 1),
 (('затягтися', 'під', 'парканами'), 1),
 (('спати', 'під', 'наркозом'), 1)]

In [50]:
def most_freq_conj_adj(corpus, n):
    "Return the most common adj-and-adj collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "ADJF" and corpus[i-1].word == "і" and corpus[i-2].tag.POS == "ADJF":
            collocs[(corpus[i-2].normal_form, corpus[i-1].word, word.normal_form)] += 1
    return collocs.most_common(n)

In [51]:
most_freq_conj_adj(corpus_t, 20)

[(('більший', 'і', 'менший'), 3),
 (('хоробрий', 'і', 'сміливий'), 2),
 (('гостинний', 'і', 'привітний'), 2),
 (('найбільший', 'і', 'найстрашніший'), 1),
 (('фантастичний', 'і', 'реальний'), 1),
 (('понурий', 'і', 'глибокий'), 1),
 (('тупий', 'і', 'байдужий'), 1),
 (('непокірний', 'і', 'гордий'), 1),
 (('волелюбний', 'і', 'сплюндрований'), 1),
 (('невідомий', 'і', 'вимріяний'), 1),
 (('найліпший', 'і', 'наймодерніший'), 1),
 (('пристосований', 'і', 'устаткований'), 1),
 (('радісний', 'і', 'святковий'), 1),
 (('експансивний', 'і', 'горластий'), 1),
 (('причепурений', 'і', 'розгальмований'), 1),
 (('цивільний', 'і', 'військовий'), 1),
 (('відкритий', 'і', 'зухвалий'), 1),
 (('реальний', 'і', 'напівфантастичний'), 1),
 (('розпливчастий', 'і', 'туманний'), 1),
 (('напружений', 'і', 'намагнетизовапий'), 1)]

In [52]:
most_freq_conj_adj(corpus_s, 20)

[(('денний', 'і', 'нічний'), 2),
 (('перший', 'і', 'останній'), 1),
 (('близький', 'і', 'дальший'), 1),
 (('вищий', 'і', 'нижчий'), 1),
 (('підступний', 'і', 'мстивий'), 1),
 (('мудрий', 'і', 'людяний'), 1),
 (('партійний', 'і', 'профспілковий'), 1),
 (('український', 'і', 'заводський'), 1),
 (('близький', 'і', 'далекий'), 1),
 (('розмлявлений', 'і', 'знудьгований'), 1),
 (('неминучий', 'і', 'незабарний'), 1),
 (('блідий', 'і', 'знекровлений'), 1),
 (('повний', 'і', 'чистий'), 1),
 (('мудрий', 'і', 'дужий'), 1)]

#### Як дослідити контексти слова?

In [53]:
def show_concordance(corpus, lemma, n):
    contexts = []
    for i in range(len(corpus)):
        if corpus[i].normal_form == lemma:
            left_context = corpus[max(0, i-5) : i]
            right_context = corpus[i+1 : min(i+6, len(corpus))]
            left = " ".join([word.word for word in left_context])
            right = " ".join([word.word for word in right_context])
            contexts.append("{:30}\t{:10}\t{}".format(left, corpus[i].word, right))
    return contexts[:n]

In [54]:
for i in show_concordance(corpus_t, "спати", 20):
    print(i)

начальник не їсть і не        	спить     	— все прибігає з поклоном
ними коїться . вдень хочеться 	спати     	, а вночі — їсти
вже ні їсти , ні              	спати     	, а другі , навпаки
до тільки їсти і тільки       	спати     	, а інші й ще
й ще краще — могли            	спати     	і тільки спати . але
— могли спати і тільки        	спати     	. але назагал , абсолютну
? ото буде хлопцям тепло      	спати     	! — чорт візьми ,
, закатованих ... ти лягатимеш	спати     	— і не зможеш заснути
боявся , нарешті , сам        	спати     	— і одружився . а
ковдру і прикинувся , що      	спить     	. в хату щось увійшло
подумували , чи не вкладатися 	спати     	. про вечерю не заїкнувся
зручно щокою до сідла .       	спить     	. не спить . лежить
сідла . спить . не            	спить     	. лежить нерухомо . заливай
заплющила очі ? либонь ,      	спить     	собі ... хай спить ...
, спить собі ... хай          	спить     	... а діди тихо бубоніли
— одтухав . потім ліг         	спати     	, 

In [55]:
for i in show_concordance(corpus_s, "спати", 20):
    print(i)

, канючливих клопотів . міцно 	спить     	під наркозом акацій , що
, совісно , люди ж            	сплять    	, тому й далі тільки
тоді черга зачіплянці сміятись ...	спить     	студент , довгов'язий смаглявець з
шелестить вода по садках .    	спить     	трудове передмістя , повите снами
ступі не влучиш ... одначе    	спати     	пора . вставши , вірунька
обережно поправила ті ноженята :	спіть     	, набігались за день .
вікнами , в мене діти         	сплять    	. — і , прихиливши
під законом , спокійно можуть 	спати     	, а ця в постійній
) , як відомо ,               	сплять    	хвостом до берега , мордою
зорями ночуватимуть . і взимку	спатимуть 	у лісі , в хутряних
скине і в чисту постіль       	спати     	вкладе , бо ж годувальник
: будьте лицарями ! солодко   	сплять    	лицарі , шкода й будити
: — громадянине , не          	спіть     	. — я не спав
прерікайтесь . гордий дуже .  	спати     	треба було в електричці ,
, бо на посту не              	сплять    	. нишком живе , не
ясно :

## А як щодо інших текстів?

Скопіюйте текст будь-якої новини. Наприклад, https://hromadske.ua/posts/ukrayina-evakuyuye-z-uhanya-razom-zi-svoyimi-gromadyanami-do-25-inozemciv-moz.

In [56]:
news = """
В Міністерстві охорони здоров’я повідомили, що Україна евакуює з карантинного міста Ухань китайської провінції Хубей 49 українців та до 25 іноземних громадян через смертельний коронавірус.
Про це на брифінгу розповів заступник міністра охорони здоров’я Віктор Ляшко.

«До 25 іноземних громадян. Хто ці особи вирішує Міністерство закордонних справ по дипломатичних каналах. Але вони усі будуть однаково проходити обсервацію, без різниці з якої країни», — зазначив чиновник.

Також він додав, що наразі МОЗ обрало два медзаклади, куди доправлять евакуйованих на 14-денний карантин.

«Сьогодні я їх не можу озвучити, але вони є, вони визначені і по них ведеться повна підготовка. Один основний, один резервний на випадок, якщо щось станеться під час моменту евакуації», — сказав Ляшко.

На брифінгу він також повідомив, що евакуацію українців з китайської провінції Хубей, яка є епіцентром спалаху коронавірусу, знову перенесли. Раніше очікувалося, що літак з українцями прибуде у середу 19 лютого, тепер його очікують у четвер, 20 лютого.

Раніше Міністерство закордонних справ Аргентини висловило подяку Україні за те, що українська влада погодилась разом зі своїми громадянами евакуювати й громадян Аргентини з китайського Уханя, епіцентру поширення коронавірусу. Скільки саме аргентинців забере український літак з Уханя, відомство не уточнило.

За останніми даними, кількість людей, інфікованих китайським коронавірусом, збільшилась до 71 331. Через вірус померли 1 775 пацієнтів. У Гонконгу зафіксували 57 випадків інфікування та один смертельний випадок.
"""

In [57]:
corpus_news = [morph.parse(word)[0] for word in tokenize_uk.tokenize_words(news)]

In [58]:
most_freq(corpus_news, 10)

[('китайський', 4),
 ('громадянин', 4),
 ('коронавірус', 4),
 ('міністерство', 3),
 ('ухань', 3),
 ('українець', 3),
 ('випадок', 3),
 ('охорона', 2),
 ('здоров’я', 2),
 ('повідомити', 2)]

In [59]:
most_freq_colloc(corpus_news, 10)

[(('китайський', 'провінція'), 2),
 (('іноземний', 'громадянин'), 2),
 (('карантинний', 'місто'), 1),
 (('смертельний', 'коронавірус'), 1),
 (('дипломатичний', 'канал'), 1),
 (('14-денний', 'карантин'), 1),
 (('повний', 'підготовка'), 1),
 (('раніший', 'міністерство'), 1),
 (('український', 'влад'), 1),
 (('китайський', 'ухань'), 1)]

#### Продовження заняття можна знайти за цим посиланням:
http://tiny.cc/0p63jz